In [5]:

# Cell 1: Import libraries
import pandas as pd


In [6]:

# Cell 2: Load the Excel file (assumes the first sheet has the data)
file_path = "Intentional Homicide Victims by counts and rates p.xls"
data = pd.read_excel(file_path, skiprows=2)

# Quick peek at the top rows
data.head()


,Iso3_code,Region,Subregion,Country,Source,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE
0,ARM,Asia,Western Asia,Armenia,CTS,by citizenship,National citizens,Male,Total,2013,Counts,35.0
1,CHE,Europe,Western Europe,Switzerland,CTS,by citizenship,National citizens,Male,Total,2013,Counts,28.0
2,COL,Americas,Latin America and the Caribbean,Colombia,CTS,by citizenship,National citizens,Male,Total,2013,Counts,15053.0
3,CZE,Europe,Eastern Europe,Czechia,CTS,by citizenship,National citizens,Male,Total,2013,Counts,69.0
4,DEU,Europe,Western Europe,Germany,CTS,by citizenship,National citizens,Male,Total,2013,Counts,455.0


In [7]:

# Cell 3: Preview conversions (we will apply on a copy later)
pd.to_numeric(data['Year'], errors='coerce')
pd.to_numeric(data['VALUE'], errors='coerce')

print("Planned conversions previewed. Actual conversion will be applied to a copy.")


Planned conversions previewed. Actual conversion will be applied to a copy.


In [8]:

# Cell 4: Basic structure checks
print("Shape:", data.shape)
print("\nColumns:")
print(list(data.columns))


Shape: (118401, 12)

Columns:
['Iso3_code', 'Region', 'Subregion', 'Country', 'Source', 'Dimension', 'Category', 'Sex', 'Age', 'Year', 'Unit of measurement', 'VALUE']


In [9]:

# Cell 5: DataFrame info
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118401 entries, 0 to 118400
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Iso3_code            118401 non-null  object 
 1   Region               118397 non-null  object 
 2   Subregion            118397 non-null  object 
 3   Country              118397 non-null  object 
 4   Source               118401 non-null  object 
 5   Dimension            118401 non-null  object 
 6   Category             118401 non-null  object 
 7   Sex                  118401 non-null  object 
 8   Age                  118401 non-null  object 
 9   Year                 118401 non-null  int64  
 10  Unit of measurement  118401 non-null  object 
 11  VALUE                118401 non-null  float64
dtypes: float64(1), int64(1), object(10)
memory usage: 10.8+ MB


In [10]:

# Cell 6: Missing values count
data.isnull().sum().sort_values(ascending=False)


Region                 4
Subregion              4
Country                4
Iso3_code              0
Source                 0
Dimension              0
Category               0
Sex                    0
Age                    0
Year                   0
Unit of measurement    0
VALUE                  0
dtype: int64

In [11]:

# Cell 7: Unique values in important columns (safe checks)
if 'Unit of measurement' in data.columns:
    print("Units of measurement (unique):", data['Unit of measurement'].dropna().unique()[:10])
if 'Dimension' in data.columns:
    print("Dimensions (unique sample):", data['Dimension'].dropna().unique()[:10])
if 'Year' in data.columns:
    print("Year range:", data['Year'].min(), "to", data['Year'].max())


Units of measurement (unique): ['Counts' 'Rate per 100,000 population']
Dimensions (unique sample): ['by citizenship' 'Total' 'by relationship to perpetrator'
 'by situational context' 'by mechanisms' 'by location']
Year range: 1990 to 2023


In [12]:

# Cell 8: Copy the original DataFrame so raw 'data' stays intact
df = data.copy()
print("Copy created. Shape:", df.shape)


Copy created. Shape: (118401, 12)


In [13]:

# Cell 9: Remove leading/trailing spaces in all object (string) columns
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.strip()

print("Stripped whitespace in text columns.")


Stripped whitespace in text columns.


In [14]:

# Cell 10: Convert 'Year' and 'VALUE' to numeric types (simple, beginner-friendly)
df['Year'] = pd.to_numeric(df['Year'], errors='coerce').astype('Int64')  # nullable integer
df['VALUE'] = pd.to_numeric(df['VALUE'], errors='coerce')               # float

print(df[['Year', 'VALUE']].dtypes)


Year       Int64
VALUE    float64
dtype: object


In [15]:

# Cell 11: Fix simple, common text issues (optional and minimal)
# Convert literal 'nan'/'NaN'/'None' strings to actual missing values
df.replace({'nan': pd.NA, 'NaN': pd.NA, 'None': pd.NA}, inplace=True)

# Keep unit labels as-is (placeholder for normalization if needed)
if 'Unit of measurement' in df.columns:
    df['Unit of measurement'] = df['Unit of measurement'].replace({
        'Rate per 100,000 population': 'Rate per 100,000 population',
        'Counts': 'Counts'
    })

print("Light label tidy completed (if applicable).")


Light label tidy completed (if applicable).


In [16]:

# Cell 12: Drop rows with missing essential fields
before = len(df)
df = df.dropna(subset=['Year', 'VALUE'])
after = len(df)

print(f"Dropped {before - after} rows with missing Year or VALUE.")


Dropped 0 rows with missing Year or VALUE.


In [17]:

# Cell 13: Remove exact duplicates to avoid double counting
before = len(df)
df = df.drop_duplicates()
after = len(df)

print(f"Removed {before - after} duplicate rows.")


Removed 2815 duplicate rows.


In [18]:

# Cell 14: Final sanity checks on the cleaned DataFrame and tiny preview
cleaned = df.copy()

print("Columns:", list(cleaned.columns))
print("Year range:", cleaned['Year'].min(), "to", cleaned['Year'].max())

if 'Unit of measurement' in cleaned.columns:
    print("Units present:", cleaned['Unit of measurement'].dropna().unique())

if 'Dimension' in cleaned.columns:
    print("Dimensions sample:", cleaned['Dimension'].dropna().unique()[:10])

print("\nPreview (first 5 rows):")
try:
    from IPython.display import display
    display(cleaned.head())
except Exception:
    print(cleaned.head())

print("\nMissing values (top columns):")
try:
    from IPython.display import display
    display(cleaned.isnull().sum().sort_values(ascending=False).head(10))
except Exception:
    print(cleaned.isnull().sum().sort_values(ascending=False).head(10))


Columns: ['Iso3_code', 'Region', 'Subregion', 'Country', 'Source', 'Dimension', 'Category', 'Sex', 'Age', 'Year', 'Unit of measurement', 'VALUE']
Year range: 1990 to 2023
Units present: ['Counts' 'Rate per 100,000 population']
Dimensions sample: ['by citizenship' 'Total' 'by relationship to perpetrator'
 'by situational context' 'by mechanisms' 'by location']

Preview (first 5 rows):


,Iso3_code,Region,Subregion,Country,Source,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE
0,ARM,Asia,Western Asia,Armenia,CTS,by citizenship,National citizens,Male,Total,2013,Counts,35.0
1,CHE,Europe,Western Europe,Switzerland,CTS,by citizenship,National citizens,Male,Total,2013,Counts,28.0
2,COL,Americas,Latin America and the Caribbean,Colombia,CTS,by citizenship,National citizens,Male,Total,2013,Counts,15053.0
3,CZE,Europe,Eastern Europe,Czechia,CTS,by citizenship,National citizens,Male,Total,2013,Counts,69.0
4,DEU,Europe,Western Europe,Germany,CTS,by citizenship,National citizens,Male,Total,2013,Counts,455.0



Missing values (top columns):


Region       4
Subregion    4
Country      4
Iso3_code    0
Source       0
Dimension    0
Category     0
Sex          0
Age          0
Year         0
dtype: int64

In [19]:

# Cell 15 (Optional): Create a core slice for easy comparisons
if {'Dimension', 'Sex', 'Age'}.issubset(cleaned.columns):
    core = cleaned[
        (cleaned['Dimension'] == 'Total') &
        (cleaned['Sex'] == 'Total') &
        (cleaned['Age'] == 'Total')
    ].copy()
    print("Core slice created. Shape:", core.shape)
    try:
        from IPython.display import display
        display(core.head())
    except Exception:
        print(core.head())
else:
    print("Core slice not created because one or more required columns are missing.")


Core slice created. Shape: (12682, 12)


,Iso3_code,Region,Subregion,Country,Source,Dimension,Category,Sex,Age,Year,Unit of measurement,VALUE
1457,ABW,Americas,Latin America and the Caribbean,Aruba,MD,Total,Total,Total,Total,1990,Counts,0.0
1458,AIA,Americas,Latin America and the Caribbean,Anguilla,WHO/NSO/SDG/PAHO,Total,Total,Total,Total,1990,Counts,0.0
1459,ARM,Asia,Western Asia,Armenia,UNSDC/TransMonEE/CTS/Covid-19/SDG,Total,Total,Total,Total,1990,Counts,178.0
1460,ATG,Americas,Latin America and the Caribbean,Antigua and Barbuda,MD/PAHO/OAS/CTS,Total,Total,Total,Total,1990,Counts,1.0
1461,AUS,Oceania,Australia and New Zealand,Australia,MD/Eurostat/GSH 2019 Revision/CTS,Total,Total,Total,Total,1990,Counts,374.0


In [20]:

# Cell 16: Save the cleaned dataset (CSV and optional Excel)
csv_path = "cleaned_intentional_homicide.csv"
xlsx_path = "cleaned_intentional_homicide.xlsx"  # optional

cleaned.to_csv(csv_path, index=False, encoding="utf-8")
print("Saved CSV to:", csv_path)

try:
    cleaned.to_excel(xlsx_path, index=False)
    print("Saved Excel to:", xlsx_path)
except Exception as e:
    print("Excel save skipped (optional). Reason:", e)


Saved CSV to: cleaned_intentional_homicide.csv
Saved Excel to: cleaned_intentional_homicide.xlsx
